In [1]:
import sys
sys.path.insert(0, '../src/')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz

import torch

import utils
from net import start_experiments

In [2]:
_A_obs = load_npz('../data/datasets/CORA_ML.npz')

val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))
assert (train_graph.toarray() == train_graph.toarray().T).all()

In [9]:
models = start_experiments(num_experiments=5,
                           experiment_root='../logs/CORA-ML/Ours',
                           train_graph=train_graph,
                           H=9,
                           optimizer=torch.optim.Adam,
                           optimizer_args={'lr': 0.1,
                                           'weight_decay': 1e-6},
                           invoke_every=4,
                           steps = 80,
                           val_edges=(test_ones, test_zeros))


Experiment_0
Step:  4/80, Loss: 7.72837, Edge-Overlap: 0.002
Step:  8/80, Loss: 6.74021, Edge-Overlap: 0.013
Step: 12/80, Loss: 5.53558, Edge-Overlap: 0.088
Step: 16/80, Loss: 4.74987, Edge-Overlap: 0.204
Step: 20/80, Loss: 4.18803, Edge-Overlap: 0.270
Step: 24/80, Loss: 3.78860, Edge-Overlap: 0.346
Step: 28/80, Loss: 3.49620, Edge-Overlap: 0.415
Step: 32/80, Loss: 3.28111, Edge-Overlap: 0.465
Step: 36/80, Loss: 3.12439, Edge-Overlap: 0.497
Step: 40/80, Loss: 3.00497, Edge-Overlap: 0.544
Step: 44/80, Loss: 2.91239, Edge-Overlap: 0.562
Step: 48/80, Loss: 2.83781, Edge-Overlap: 0.590
Step: 52/80, Loss: 2.77704, Edge-Overlap: 0.609
Step: 56/80, Loss: 2.72638, Edge-Overlap: 0.627
Step: 60/80, Loss: 2.68404, Edge-Overlap: 0.639
Step: 64/80, Loss: 2.64832, Edge-Overlap: 0.658
Step: 68/80, Loss: 2.61754, Edge-Overlap: 0.658
Step: 72/80, Loss: 2.59096, Edge-Overlap: 0.675
Step: 76/80, Loss: 2.56771, Edge-Overlap: 0.685
Step: 80/80, Loss: 2.54719, Edge-Overlap: 0.694

Experiment_1
Step:  4/80,